In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
config_file = "configs/train_rdd2020_server.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-04-22 17:52:27,793 SSD INFO: Loaded configuration file configs/train_rdd2020_server.yaml
2021-04-22 17:52:27,795 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'resnet18'
        PRETRAINED: True
        #OUT_CHANNELS: [512, 1024, 2048, 1024, 1024, 512] # resnet152
        OUT_CHANNELS: [128, 256, 512, 512, 256, 256] # resnet18
        #OUT_CHANNELS: [512, 1024, 512, 512, 256, 256] # model from assignment 4
        INPUT_CHANNELS: 3
    PRIORS:
        # Image size [300, 300]
        #FEATURE_MAPS: [[38, 38], [19, 19], [10, 10], [5, 5], [3, 3], [1, 1]]
        #STRIDES: [[8, 8], [16, 16], [32, 32], [64, 64], [100, 100], [300, 300]]
        #MIN_SIZES: [[30, 30], [60, 60], [111, 111], [162, 162], [213, 213], [264, 264]]
        #MAX_SIZES: [[60, 60], [111, 111], [162, 162], [213, 213], [264, 264], [315, 315]]

        # Image size [600, 600]
        #FEATURE_MAPS: [[75, 75], [38, 38], [19, 19], [10, 10], [5, 5], [3, 3]]
        #MIN_SIZES: [[60, 60], [120, 120], [

In [ ]:
model = start_train(cfg)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/fredralm/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



Detector initialized. Total Number of params:  21.96M
Backbone number of parameters: 21.13M
SSD Head number of parameters: 829.7K
2021-04-22 17:52:33,921 SSD.trainer INFO: No checkpoint found.
Dataset loaded. Subset: train, number of images: 8536
2021-04-22 17:52:34,065 SSD.trainer INFO: Start training ...


/home/fredralm/home/TDT4265-StarterCode/Project/SSD/ssd/data/transforms/transforms.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/fredralm/home/TDT4265-StarterCode/Project/SSD/ssd/data/transforms/transforms.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/fredralm/home/TDT4265-StarterCode/Project/SSD/ssd/data/transforms/transforms.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of list

2021-04-22 17:52:44,595 SSD.trainer INFO: iter: 000010, lr: 0.00050, total_loss: 16.846 (16.846), reg_loss: 3.424 (3.424), cls_loss: 13.421 (13.421), time: 0.902 (0.902), eta: 1 day, 6:03:17, mem: 3849M
2021-04-22 17:52:46,890 SSD.trainer INFO: iter: 000020, lr: 0.00050, total_loss: 13.507 (15.176), reg_loss: 3.203 (3.314), cls_loss: 10.304 (11.863), time: 0.230 (0.566), eta: 18:51:07, mem: 3849M
2021-04-22 17:52:49,167 SSD.trainer INFO: iter: 000030, lr: 0.00050, total_loss: 11.736 (14.030), reg_loss: 3.066 (3.231), cls_loss: 8.670 (10.798), time: 0.228 (0.453), eta: 15:05:45, mem: 3849M
2021-04-22 17:52:51,441 SSD.trainer INFO: iter: 000040, lr: 0.00050, total_loss: 10.449 (13.134), reg_loss: 2.931 (3.156), cls_loss: 7.518 (9.978), time: 0.227 (0.397), eta: 13:12:53, mem: 3849M
2021-04-22 17:52:53,721 SSD.trainer INFO: iter: 000050, lr: 0.00050, total_loss: 9.524 (12.412), reg_loss: 2.842 (3.093), cls_loss: 6.682 (9.319), time: 0.228 (0.363), eta: 12:05:26, mem: 3849M
2021-04-22 17:5

100%|██████████| 366/366 [23:46<00:00,  3.90s/it]


...

2021-04-22 19:19:48,452 SSD.inference INFO: mAP: 0.1785
D00             : 0.1094
D10             : 0.0818
D20             : 0.4323
D40             : 0.0905

2021-04-22 19:19:50,730 SSD.trainer INFO: iter: 010010, lr: 0.00050, total_loss: 4.552 (5.065), reg_loss: 1.713 (1.997), cls_loss: 2.839 (3.068), time: 135.053 (0.523), eta: 15:58:43, mem: 3849M
2021-04-22 19:19:53,119 SSD.trainer INFO: iter: 010020, lr: 0.00050, total_loss: 4.415 (5.064), reg_loss: 1.639 (1.996), cls_loss: 2.775 (3.068), time: 0.239 (0.523), eta: 15:58:07, mem: 3849M
2021-04-22 19:19:55,474 SSD.trainer INFO: iter: 010030, lr: 0.00050, total_loss: 4.458 (5.064), reg_loss: 1.679 (1.996), cls_loss: 2.779 (3.068), time: 0.235 (0.522), eta: 15:57:30, mem: 3849M
2021-04-22 19:19:57,802 SSD.trainer INFO: iter: 010040, lr: 0.00050, total_loss: 4.623 (5.063), reg_loss: 1.731 (1.996), cls_loss: 2.892 (3.067), time: 0.233 (0.522), eta: 15:56:53, mem: 3849M
2021-04-22 19:20:00,133 SSD.trainer INFO: iter: 010050, lr: 0.00050, 

100%|██████████| 366/366 [23:06<00:00,  3.79s/it]


2021-04-22 20:03:09,198 SSD.inference INFO: mAP: 0.2263
D00             : 0.1461
D10             : 0.0720
D20             : 0.4853
D40             : 0.2018

2021-04-22 20:03:11,467 SSD.trainer INFO: iter: 015010, lr: 0.00050, total_loss: 4.352 (4.872), reg_loss: 1.629 (1.897), cls_loss: 2.723 (2.976), time: 139.105 (0.522), eta: 15:13:29, mem: 3849M
2021-04-22 20:03:13,788 SSD.trainer INFO: iter: 015020, lr: 0.00050, total_loss: 4.426 (4.872), reg_loss: 1.657 (1.897), cls_loss: 2.769 (2.975), time: 0.232 (0.522), eta: 15:13:03, mem: 3849M
2021-04-22 20:03:16,115 SSD.trainer INFO: iter: 015030, lr: 0.00050, total_loss: 4.388 (4.872), reg_loss: 1.677 (1.896), cls_loss: 2.711 (2.975), time: 0.233 (0.522), eta: 15:12:38, mem: 3849M
2021-04-22 20:03:18,446 SSD.trainer INFO: iter: 015040, lr: 0.00050, total_loss: 4.414 (4.871), reg_loss: 1.647 (1.896), cls_loss: 2.766 (2.975), time: 0.233 (0.521), eta: 15:12:13, mem: 3849M
2021-04-22 20:03:20,767 SSD.trainer INFO: iter: 015050, lr: 0.00050, 

100%|██████████| 366/366 [21:36<00:00,  3.54s/it]


2021-04-22 20:44:55,215 SSD.inference INFO: mAP: 0.2241
D00             : 0.1658
D10             : 0.0942
D20             : 0.4905
D40             : 0.1460



In [5]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)

2021-04-21 18:09:34,587 SSD INFO: Start evaluating...


NameError: name 'model' is not defined